In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot_futures import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
settings = load_settings(user)
s = settings['symbol']
settings

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [ ]:
n_days_load = 7
agg_trades = await bot.load_trades(s, n_days_load)
adf = agg_trades

In [ ]:
adf = agg_trades

In [ ]:
adf.price.iloc[::100].plot()

In [ ]:
# modify settings
settings['entry_amount'] = 0.001
settings['leverage'] = 100
settings['markup'] = 0.00155
settings['flashcrash_factor'] = 0.001
settings['ema_span'] = 1000
settings['enter_long'] = True
settings['enter_shrt'] = True
settings

In [ ]:
trades, adf = backtest(adf, settings)

tdf = pd.DataFrame(trades).set_index('agg_id')
bdf = tdf[tdf.side == 'buy']
sdf = tdf[tdf.side == 'sel']
tdf

In [ ]:
realized_pnl_sum = tdf.realized_pnl.sum()
fee_sum = tdf.fee.sum()
print('realized pnl sum', realized_pnl_sum)
print('fee sum', fee_sum)
print('pnl - fee', realized_pnl_sum - fee_sum)
print('pct fees paid', fee_sum / realized_pnl_sum)

In [ ]:
print('pos amount counts')
tdf.amount.value_counts()

In [ ]:
min_needed_margin = tdf.margin_cost.max()
print('min needed margin', min_needed_margin)

In [ ]:
(tdf.realized_pnl - tdf.fee).cumsum().plot()

In [ ]:
tdf.amount.plot()

In [ ]:
gain = (realized_pnl_sum - fee_sum + min_needed_margin) / min_needed_margin
n_days = (tdf.timestamp.max() - tdf.timestamp.min()) / 1000 / 60 / 60 / 24
adg = gain ** (1 / n_days)
average_daily_abs_profit = realized_pnl_sum / n_days
print('min_needed_margin', min_needed_margin)
print('realized_pnl_sum', realized_pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average daily gain', adg)
print('average_daily_abs_profit', average_daily_abs_profit)

In [ ]:
# find location of biggest position size
idx = tdf.amount.abs().idxmax()
idxloc = tdf.index.get_loc(idx)
tdf.iloc[idxloc - 15:idxloc + 2].index[0]

In [ ]:
# plot
istart = tdf.iloc[idxloc - 15:idxloc + 2].index[0]
iend = istart + 300000
bid_name = f"entry_bid_{settings['ema_span']}_{settings['flashcrash_factor'] * 1000}".replace('.', '_')
ask_name = f"entry_ask_{settings['ema_span']}_{settings['flashcrash_factor'] * 1000}".replace('.', '_')
adf.loc[istart:iend][['price', bid_name, ask_name]].plot()
bdfc = bdf[(bdf.index >= istart) & (bdf.index < iend)]
sdfc = sdf[(sdf.index >= istart) & (sdf.index < iend)]

bdfc.price.plot(style='bo')
sdfc.price.plot(style='ro')

In [ ]:
bdfc

In [ ]:
# for testing multiple settings

In [ ]:
rs = {}

In [ ]:
vs = [(300, 100000, 0),   # min max precision ema_span
      (0.0001, 0.004, 5), # min max precision flashcrash_factor
      (0.0009, 0.006, 5), # min max precision markup
      (40, 125, 0)]       # min max precision leverage

best = tuple([round((v[0] + v[1]) / 2, v[2]) for v in vs])


# mutate settings randomly
def get_candidate(best, m=0.2):
    new_candidate = []
    for i, v in enumerate(vs):
        new_val = best[i] + (np.random.random() - 0.5) * (v[1] - v[0]) * max(0.0001, m)
        new_candidate.append(round(max(min(new_val, v[1]), v[0]), v[2]))
    return tuple(new_candidate)
best

In [ ]:
best_gain = 0
key = best
ks = 100
k = 0
margin_cost_limit = 250

In [ ]:
conditions = [
    lambda r: r['n_trades'] > 5000
]

In [ ]:
while k <= ks:
    k += 1
    adf = agg_trades
    settings['ema_span'] = key[0]
    settings['flashcrash_factor'] = key[1]
    settings['markup'] = key[2]
    settings['leverage'] = key[3]
    if key in rs:
        print('skipping', key)
        key = get_candidate(best)
        continue
    print(k, best, key)
    trades, adf = backtest(adf, settings, margin_cost_limit=margin_cost_limit)
    tdf = pd.DataFrame(trades).set_index('agg_id')
    result = {'net_pnl': tdf.realized_pnl.sum() - tdf.fee.sum()}
    result['amount_max'] = tdf.amount.max()
    result['amount_min'] = tdf.amount.min()
    result['amount_abs_max'] = tdf.amount.abs().max()
    result['amount_abs_sum'] = tdf.amount.abs().sum()
    result['n_trades'] = len(trades)
    result['margin_cost_max'] = tdf.margin_cost.max()
    result['gain'] = (result['net_pnl'] + result['margin_cost_max']) / result['margin_cost_max']
    print('\n', key, list(map(lambda x: round(x, 6), result.values())))
    rs[key] = result
    if result['gain'] > best_gain:
        if all([condition(result) for condition in conditions]):
            best = key
            best_gain = result['gain']
            print('new best', best)
    key = get_candidate(best, m=(1 - k / ks))
    

In [ ]:
rdf = pd.DataFrame(rs).T
rdfs = rdf.sort_values('gain', ascending=[False])
rdfs.head(30)